In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import openpyexcel
from openpyexcel import workbook, load_workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyexcel.utils import get_column_letter
from itertools import islice
from os import listdir
import re
import csv
import datetime
import pickle

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)
pd.options.display.float_format = '{:.0f}'.format

In [4]:
company = pd.read_csv('CSV/company_ws.csv',index_col=0)
company

,Balance,Income,Cash
ADBE1.xlsx,Consolidated Balance Sheets,Consolidated Statements of Inco,Consolidated Statements of Cash
ADBE2.xlsx,Consolidated Balance Sheets,Consolidated Statements of Inco,Consolidated Statements of Cash
ADBE3.xlsx,Consolidated Balance Sheets,Consolidated Statements of Inco,Consolidated Statements of Cash
ADBE4.xlsx,Consolidated Balance Sheets,Consolidated Statements of Inco,Consolidated Statements of Cash
ADBE5.xlsx,Consolidated_Balance_Sheets,Consolidated_Statements_of_Inc,Consolidated_Statements_of_Cas
AME1.xlsx,Consolidated Balance Sheet,Consolidated Statement of Incom,Consolidated Statement of Cash
AME2.xlsx,Consolidated Balance Sheet,Consolidated Statement of Incom,Consolidated Statement of Cash
AME3.xlsx,Consolidated Balance Sheet,Consolidated Statement of Incom,Consolidated Statement of Cash
AME4.xlsx,Consolidated Balance Sheet,Consolidated Statement of Incom,Consolidated Statement of Cash
AME5.xlsx,Consolidated_Balance_Sheet,Consolidated_Statement_of_Inco,Consolidated_Statement_of_Cash


In [5]:
def clean_table (table):
    
    df = table.copy()  
    
    df.replace(u'\xa0','', regex=True, inplace=True)
    
    # Replace empty string with nan
    df.replace("^$",np.nan, regex=True, inplace=True)
    
    # Remove [n]
    
    for i in range(1,len(df.columns)):
      if df[df.columns[i]].dtype == object:
        df[df.columns[i]].replace("^\[\d\]", np.nan, regex=True, inplace=True)

    # Remove footnotes embedded in table
    
    df.drop(df.loc[df[df.columns[0]].str.contains(r"^\[\d?\]$", case=False, regex=True,na=False)].index,inplace=True)
    
    # Replace nan in first column with ''
    
    df[df.columns[0]].replace(np.nan,'',inplace=True)
    
    #Remove blank columns and rows
    
    df.dropna(axis=1, how='all', inplace=True)
    df.dropna(axis=0, how='all', inplace=True)
    
    if len(df.columns) > 5:
      # Take the first and the last 3 columns
        
      a = df.iloc[:,:1]
      b = df.iloc[:,-3:]
      df = pd.concat([a,b], axis=1)
    
    #Remove comments in brankets eg (Notes 1) and spurious characters
    
    df[df.columns[0]].replace(r"[\s]?\(.*\)", "", regex=True, inplace=True)
    df[df.columns[0]].replace(r"â€™","", regex=True, inplace=True)
    
    # Deal with situation where column labels are of type date time. Extract year and convert to str
  
    col = list(df.columns[0:])
    
    for i in range(0,len(col)):
      if type(col[i]) is datetime.datetime :
        col[i] = col[i].strftime('%Y')
    
    # Set columns to extraxted year which will be of type string
    # If we don't do this, the following if condition will not work.
    
    df.columns = col
        
    # If none of the columns contain a 4 digit number to signify year then take column headings from first row
    
    if df.columns.str.contains(r"\d\d\d\d", case=False, regex=True,na=False).sum() == 0:
        
      df.iloc[0].fillna('', axis=0, inplace=True)
      
      # Deal with the situation where the first row is read in as date time
    
      for i in range(0,len(df.iloc[0])):
        if type(df.iloc[0][i]) is datetime.datetime :
          df.iloc[0][i] = df.iloc[0][i].strftime('%Y')
            
      df.iloc[0] = df.iloc[0].astype('string')
      col = list(df.iloc[0])
    
      # Drop the first row
    
      df.drop(index=df.index[0], axis=0, inplace=True)
        
    col[0] = 'Index'
    
    # Extraxt year if there is a match
    
    for i in range(1,len(col)):
      res = re.search('(20[0-9][0-9])', col[i])

      if res:
        col[i] = res.group(0)
    
    df.columns = col  
    
    # Remove footnotes embedded in table
    
    df.drop(df.loc[df[df.columns[0]].str.contains(r"^\[\d?\]$", case=False, regex=True,na=False)].index,inplace=True)

    df.dropna(axis=1, how='all', inplace=True)
    df.dropna(axis=0, how='all', inplace=True)
    
     # Make sure that first column doesn't have any nulls
  
    df[df.columns[0]].fillna('', inplace=True)
    
    # Set index to first column
    
    df.set_index(df.columns[0], inplace=True)
    
    #Replace single quote with nan
    
    df.replace("'",np.nan, inplace=True)
    
    # Remove rows which contain otherwise specified
    
    df = df.loc[~df.index.str.contains(rf"unless otherwise",case=False, regex=True,na=False)]

    # Show first row as a validation check
    
    print(pd.DataFrame(df.iloc[0]).T)
    
    return df

In [31]:
def get_line_items(t):
 
  
  heading = []
  line_item = []
    
  for index, row in t.iterrows():
    
    if row.replace('', np.nan).isna().all():
      # Blank row, so assume heading
    
      heading.append(index)
    else:    
      line_item.append(index)

  return {'heading' : heading,
          'line_item' : line_item}

In [6]:
def get_spreadsheet(company,ticker,file, type):
    
  rootdir = "C:/Users/priye/Desktop/Dissertation/DATA/"

  dirname = rootdir + ticker
  path = dirname + '/' + file

  xl_file = pd.ExcelFile(path)
  table = xl_file.parse(company.loc[file,type],parse_dates=False)

  return table

In [70]:
rootdir = "C:/Users/priye/Desktop/Dissertation/DATA/"

headings = []
line_items = []
balance_sheets = {}

for d in listdir(rootdir):
    
  sheets = []
  dirname = rootdir + d
  for f in listdir(dirname):
    if f.endswith('xlsx'):
      print(f)
      table = get_spreadsheet(company,d,f,'Balance')
      t = clean_table(table)
      sheets.append(t)
        
      l_dict = get_line_items(t)      
      headings = headings + l_dict['heading']
      line_items = line_items + l_dict['line_item']
  balance_sheets[d] = sheets

# Remove duplicates from headings and line items

unique_list = set(headings)
headings = list(unique_list)

unique_list = set(line_items)
line_items = list(unique_list)

table_labels = {'headings' : headings,
                'line_items' : line_items}


ADBE1.xlsx
                2022 2021
Current assets:  NaN  NaN
ADBE2.xlsx
                2020 2019
Current assets:  NaN  NaN
ADBE3.xlsx
                2018 2017
Current assets:  NaN  NaN
ADBE4.xlsx
                2016 2015
Current assets:  NaN  NaN
ADBE5.xlsx
                2014 2013
Current assets:  NaN  NaN
ADM1.xlsx
               2022 2021
Current Assets  NaN  NaN
ADM2.xlsx
                2020  2019
Current Assets   NaN   NaN
ADM3.xlsx
                2018  2017
Current Assets   NaN   NaN
ADM4.xlsx
                2018  2017
Current Assets   NaN   NaN
ADM5.xlsx
                2014  2013
Current Assets   NaN   NaN
AME1.xlsx
                2022 2021
Current assets:  NaN  NaN
AME2.xlsx
                 2020  2019
Current assets:   NaN   NaN
AME3.xlsx
                 2018  2017
Current assets:   NaN   NaN
AME4.xlsx
                2016 2015
Current assets:  NaN  NaN
AME5.xlsx
                2014 2013
Current assets:  NaN  NaN
AMGN1.xlsx
                2022 2021
Current assets

       2022 2021
ASSETS  NaN  NaN
MSI2.xlsx
        2020  2019
ASSETS   NaN   NaN
MSI3.xlsx
        2018  2017
ASSETS   NaN   NaN
MSI4.xlsx
        2016  2015
ASSETS   NaN   NaN
MSI5.xlsx
        2014  2013
ASSETS   NaN   NaN
NKE1.xlsx
                2022 2021
Current assets:  NaN  NaN
NKE2.xlsx
                2020 2019
Current assets:  NaN  NaN
NKE3.xlsx
                2018 2017
Current assets:  NaN  NaN
NKE4.xlsx
                2016 2015
Current assets:  NaN  NaN
NKE5.xlsx
                2014 2013
Current assets:  NaN  NaN
ORCL1.xlsx
                2022 2021
Current assets:  NaN  NaN
ORCL2.xlsx
                 2020  2019
Current assets:   NaN   NaN
ORCL3.xlsx
                2018 2017
Current assets:  NaN  NaN
ORCL4.xlsx
                2016 2015
Current assets:  NaN  NaN
ORCL5.xlsx
                2014 2013
Current assets:  NaN  NaN
PCAR1.xlsx
       2022 2021
ASSETS  NaN  NaN
PCAR2.xlsx
       2020 2019
ASSETS  NaN  NaN
PCAR3.xlsx
        2018  2017
ASSETS   NaN   NaN
PCAR4.

In [61]:
def save_line_items(table_labels):
    
  with open('CSV/balance_headings.csv', 'w',encoding='utf-8') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(table_labels['headings'])
    
  with open('CSV/balance_line_items.csv', 'w',encoding='utf-8') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(table_labels['line_items'])

In [6]:
def save_balance_sheets(balance_sheets):
  with open('pickle/balance_sheets.pkl', 'wb') as fp:
    pickle.dump(balance_sheets, fp)

In [7]:
def save_table_labels(table_labels):

  with open('pickle/cash_table_labels.pkl', 'wb') as fp:
    pickle.dump(table_labels, fp)

In [64]:
save_line_items(table_labels)

In [65]:
save_table_labels(table_labels)

In [66]:
save_balance_sheets(balance_sheets)